# 1. Tokenisation

In deze notebook zijn de commentaren van de enquête ingelezen in en wordt tokenisation toegepast: het splitsen van de tekst in aparte lemma's. Daarnaast worden ook de sentimentwaarden uit Speed en Brysbaert en de demografische factoren uit de canonenquête toegevoegd aan de lemma's.

## 1.1 Imports

Hieronder worden alle gebruikte modules opgesomd en geladen om de lemma's uit de canonenquête te trekken. Hiervoor wordt vooral gebruikt gemaakt van spaCy en Pandas.

[spaCy](https://spacy.io/) is een bekende package om tekst te parsen en verwerken. Aan de hand van spaCy is het mogelijk om de woorden op te splitsen en te lemmatiseren. Dit onderzoek maakt gebruik van Bram Vanroys [load_spacy](https://github.com/BramVanroy/spacy_download) dat automatisch het model downloadt.

[pandas](https://pandas.pydata.org/docs/index.html) is een package die gebruikt wordt in Python bij het verwerken van gegevens in tabellarische vorm. Hierdoor is het mogelijk om de CSV bestanden te laden, de lemma's eruit te halen en de verschillende DataFrames samen te brengen.

In [1]:
import pandas as pd
from spacy_download import load_spacy
import os


nlp = load_spacy("nl_core_news_lg", exclude=["parser", "tagger"])
nlp.add_pipe("sentencizer")
nlp

## 1.2 Dataset inladen

De dataset uit de canonenquête is opgekuist aan de hand van [OpenRefine](https://openrefine.org/) en wordt geladen in Python. Hier zijn ook al een aantal niet-Nederlandse woorden vertaald naar het Nederlands. Daarnaast zijn ook de antwoorden van niet-betrouwbare respondenten verwijderd die genormaliseerd zijn op 100 jaar. 

In [20]:
df = pd.read_csv("data/Data-Anon-cleaned.csv")

## 1.3 Tokenisation

Deze onderstaande code neemt alle open commentaren uit de enquête, verwerkt deze tot lemma's en linkt ze aan de desbetreffende persoon.
Per lemma wordt het zinsnummer, lemmanummer, respondent-ID, part of speech en het woord in originele vorm bijgehouden.

De canonenquête bevatte 7 toelichtingsvragen en 2 open vragen:
- Q9: De aandacht voor de canon in voortgezet/secundair onderwijs.
- Q10: Leerlingen in het voortgezet/secundair onderwijs moeten enkele klassiekers lezen.
- Q11: Leerlingen in het voortgezet/secundair onderwijs moeten een elementaire kennis aangeboden krijgen van Nederlandstalige literatuurgeschiedenis
- Q12: Een Nederlandstalige literaire canon moet streven naar genderdiversiteit.
- Q14: Auteurs van verschillende culturele achtergronden moeten vertegenwoordigd zijn in de canon.
- Q15: Er kunnen meerdere Nederlandstalige literaire canons naast elkaar bestaan.
- Q16: Welke teksten uit de voormalige koloniale gebieden zouden in de canon moeten?
- Q17: Welke teksten uit de kinder- en jeugdliteratuur zouden in de canon moeten?
- QI: Heb je ideeën voor projecten die de canon in de kijker kunnen zetten?
- QC: Eventuele opmerkingen

Onderstaande code bundelt deze vragen samen waarbij nog steeds iedere vraag, user en sentence number herkenbaar zijn in de dataframe.

In [21]:
#De volgende code sorteert de open vragen uit de enquête en voegt ze toe aan een dictionary. 
ELAB = {question for question in df if "ELAB" in question}
ELAB.add("QPERS_IDEAS")
ELAB.add("QPERS_COMPLAIN")

rows = [] # een lege lijst om te vullen met de lemma's

#Via een function is het mogelijk om voor iedere open vraag dezelfde code te gebruiken waarin alles gecombineerd wordt
def add_row(question_prefix, row, sentence_number, token_number, token):
    identifier_prefix = question_prefix.replace('ELAB', '').replace('QPERS_IDEAS', 'QI_').replace('QPERS_COMPLAIN', 'QC_')
    question_prefix2 = question_prefix.replace('_ELAB', '').replace('QPERS_IDEAS', 'QI').replace('QPERS_COMPLAIN', 'QC')
    #new_row element bepaalt de structuur van hoe de lemma's opgeslaan worden
    new_row = {
        "identifier": identifier_prefix + row["ResponseId"] + '_' + str(sentence_number) + '_' + str(token_number),
        "questions": question_prefix2,
        "response_id": row["ResponseId"],
        "sentence_number": sentence_number,
        "token_number": token_number,
        "token": token.text,
        "lemma": token.lemma_,
        "pos": token.pos_
    }
    rows.append(new_row)
    
#Onderstaande loop gaat over iedere vraag in de vooraf gedefinieerde ELAB-lijst met open-commentaarvragen
for question in ELAB:
    #Lege rijen worden met deze code overgeslaan
    for index, row in df.iterrows():
        if pd.isnull(row[str(question)]):
            continue
        #Via de NLP-module worden de lemma's uit het woord afgeleid
        doc = nlp(row[str(question)])
        for sentence_number, sentence in enumerate(doc.sents): #via enumerate worden het aantal zinnen per respondent bijgehouden
            for token_number, token in enumerate (sentence):   #Hetzelfde wordt hier gedaan per woord
                if "ELAB" in question or "IDEAS" in question or "COMPLAIN" in question:
                    add_row(question, row, sentence_number, token_number, token)
                else:
                    continue

#Nadat het programma door alle vragen is gegaan, worden alle lemma's opgeslaan in een dataframe               
large_df = pd.DataFrame(rows)

# Sentimentwaarden toevoegen aan de dataset

De dataset van Speed en Brysbaert (2023) wordt geladen en gecombineerd aan de lemma's uit de canonenquête.
Niet ieder lemma zal een match vinden in deze dataset, waardoor er dataverlies zal optreden. Het is ook niet nuttig om de woorden die geen sentimentwaarden hebben te integreren in de dataset. Hierdoor zal onderstaande linken op basis van het lemma dat terug te vinden is in Speed en Brysbaert.

In [23]:
right_df = pd.read_excel('data/SpeedBrysbaertEmotionNorms.xlsx')
left_df = large_df

merged_df = left_df.merge(right_df, left_on='lemma', right_on='Word', how='left')
merged_df = merged_df.drop(columns=['Word'])

# Demografische gegevens toevoegen

Via onderstaande code worden de geanonimiseerde demografische gegevens opnieuw gelinkt aan ieder lemma aan de hand van de response_id. Hierdoor is het mogelijk om te kijken of de demografische factoren een voorspellende factor kunnen hebben op de sentimentwaarden.

Deze data worden dan opgeslagen in een CSV-document die dan opgeladen wordt in R om regressiemodellen te bouwen.

In [24]:
df = pd.read_csv('data/data_anon.csv') #de data van het CRR wordt geladen in een andere dataframe
#via een subset dataframe worden enkel response_ID en de demografische gegevens geladen voor een snellere verwerking
demographics_subset = df[['ResponseId', 'QPERS_GENDER', 'QPERS_LANG', 'QPERS_EDU', 'QPERS_LING_JOB', 'QPERS_BIRTH_COUNTRY', 'QPERS_READING', 'QPERS_AGE']]
#de kolom van de originele data wordt hernoemd zodat deze overeenkomt met de vorige code.
demographics_subset.rename(columns={'ResponseId': 'response_id'}, inplace=True)
demo_merged_df = pd.merge(merged_df, demographics_subset, on='response_id')
#Rijen waar er geen sentimentwaarden voor bestaan (denk aan "een") worden overgeslaan.
if not os.path.exists('sorted_data'): #Dit maakt een map aan indien er geen zou bestaan
    os.makedirs('sorted_data')
merged_df_final = demo_merged_df.dropna()
merged_df_final.to_csv('sorted_data/Combined_Ratings.csv') #de gecombineerde data wordt opgeslaan

/var/folders/dv/bbqr68nn3_j5swj_l880wx_40000gn/T/ipykernel_51325/3326229177.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographics_subset.rename(columns={'ResponseId': 'response_id'}, inplace=True)
